In [95]:
import glob
import os
import math
import random
import pathlib
import os
import fastai
import pandas as pd
import numpy as np
#print(os.listdir("../input"))
from PIL import Image
#from fastai.vision import *
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications
from sklearn.model_selection import train_test_split
os.getcwd()

'/home/sargam/BERKELEY/WIDS_DATATHON'

In [96]:
#work_dir = Path('/kaggle/working/')
#path = Path('../input')

train = 'train_images/train_images'
test =  'leaderboard_test_data/leaderboard_test_data'
holdout = 'leaderboard_holdout_data/leaderboard_holdout_data'
sample_sub = 'SampleSubmission.csv'
train_label = 'traininglabels.csv'

train_label_data = pd.read_csv(train_label)
train_label_data = train_label_data[~((train_label_data['score'] < 0.5) & (train_label_data['has_oilpalm'] == 0))]

plant_list = train_label_data[train_label_data['has_oilpalm'] == 1]['image_id'].tolist()
noplant_list = train_label_data[train_label_data['has_oilpalm'] == 0]['image_id'].tolist()

num_plant = len(plant_list)
num_noplant = len(noplant_list)

random.seed(22)
val_plant = np.random.choice(num_plant, 50, replace = False)
val_noplant = np.random.choice(num_noplant, 100, replace = False)
train_label_data.shape

(15161, 3)

In [97]:
# move validation files to validation set
for i in val_plant:
    os.rename('train_images/' + plant_list[i], "images/val/plant/" + plant_list[i])
for i in val_noplant:
    os.rename("train_images/" + noplant_list[i], "images/val/no_plant/" + noplant_list[i]) 
    

In [98]:
# move the rest plant to train dataset
val_images = os.listdir('images/val/plant') + os.listdir('images/val/no_plant')
remain_images = list(set(list(train_label_data['image_id'])) - set(val_images))

train_label_data = train_label_data[train_label_data['image_id'].isin(remain_images)]
plant_list = train_label_data[train_label_data['has_oilpalm'] == 1]['image_id'].tolist()
noplant_list = train_label_data[train_label_data['has_oilpalm'] == 0]['image_id'].tolist()

num_plant = len(plant_list)
num_noplant = len(noplant_list)

random.seed(22)
train_plant = np.random.choice(num_plant, 250, replace = False)
train_noplant = np.random.choice(num_noplant, 500, replace = False)
for i in train_plant:
    os.rename('train_images/' + plant_list[i], "images/train/plant/" + plant_list[i])
for i in train_noplant:
    os.rename('train_images/' + noplant_list[i], "images/train/no_plant/" + noplant_list[i])

In [100]:
# Input image size
img_width = 256
img_height = 256
epochs = 50
batch_size = 25

# save file name
top_model_weights_path = 'bottleneck_fc_model.h5'

# image dir
train_data_dir = 'images/train'
validation_data_dir = 'images/val'
pred_data_dir = 'images/pred'
nb_train_samples = len(glob.glob('images/train/plant/*.jpg')) + len(glob.glob('images/train/non/*.jpg'))
nb_validation_samples = len(glob.glob('images/val/plant/*.jpg'))+len(glob.glob('images/val/non/*.jpg'))

In [115]:
# DataGenerator
datagen = ImageDataGenerator(rescale = 1./255)

# build InceptionV3 network
modelInceptionV3 = applications.InceptionV3(include_top = False, weights = 'imagenet')

generator_train = datagen.flow_from_directory(
    train_data_dir,
    color_mode = "rgb",
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = True)

bottleneck_features_train = modelInceptionV3.predict_generator(generator_train, 
                                                               nb_train_samples // batch_size)
np.save('bottleneck_features_train.npy', bottleneck_features_train)
bottleneck_features_train.shape

Found 750 images belonging to 2 classes.


(250, 6, 6, 2048)

In [116]:
# ValDataGenerator
generator_val = datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    color_mode = "rgb",
    batch_size = batch_size,
    class_mode = 'binary',
    shuffle = False) # our data will be in order, so all first 1000 images will be cats, then 1000 dogs

bottleneck_features_validation = modelInceptionV3.predict_generator(generator_val, 
                                                                    nb_validation_samples // batch_size)

np.save('bottleneck_features_validation.npy', bottleneck_features_validation)
bottleneck_features_validation.shape

Found 150 images belonging to 2 classes.


(50, 6, 6, 2048)

In [125]:
train_data = np.load('bottleneck_features_train.npy')
train_labels = generator_train.classes

validation_data = np.load('bottleneck_features_validation.npy')
validation_labels = generator_val.classes

In [127]:
len(train_labels)

750

In [128]:
model = Sequential()
model.add(Flatten(input_shape = train_data.shape[1:]))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [129]:
# try different optimizer
# change learning rate
from keras import optimizers
adams = optimizers.Adam(lr = 0.001)
#sgd = optimizers.SGD(lr=0.05, clipnorm=0.5)
#sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

model = Sequential()
model.add(Flatten(input_shape = train_data.shape[1:]))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer=adams,
              loss = 'binary_crossentropy', metrics = ['accuracy'])
generator_train.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [133]:
print(len(train_labels), len(train_data), len(validation_labels), len(validation_data))


750 250 150 50


In [131]:
model.fit(train_data, 
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_data = (validation_data, 
                             validation_labels))
model.save_weights(top_model_weights_path)

ValueError: Input arrays should have the same number of samples as target arrays. Found 250 input samples and 750 target samples.